In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyTelegramBotAPI requests
!pip install openai
!pip install transformers[image-processing]
!pip install easyocr


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import telebot
import requests
from PIL import Image
import io
import easyocr

bot_token = '6272552984:AAE_-kelmQhURmoRNgGXZ1lopZKDoomO254'

reader = easyocr.Reader(['en'])  # Set the language(s) for OCR

bot = telebot.TeleBot(bot_token)

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    # Get the photo file ID and file path
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    file_path = file_info.file_path

    # Download the photo
    photo_url = f"https://api.telegram.org/file/bot{bot_token}/{file_path}"
    photo_data = requests.get(photo_url).content
    
    try:
        # Open the photo using PIL
        photo = Image.open(io.BytesIO(photo_data))
        
        # Check if the image format is supported
        supported_formats = ["JPEG", "PNG", "GIF"]
        if photo.format not in supported_formats:
            raise ValueError("Unsupported image format")

        # Convert the photo to text using OCR
        results = reader.readtext(photo)

        # Extract the texts from OCR results0
        texts = [result[1] for result in results]
        
        # Reply to the user with the extracted texts or a custom message
        if texts:
            reply_text = '\n'.join(texts)
        else:
            reply_text = "No text found in the image. Please make sure the image contains clear and recognizable text."
        bot.reply_to(message, reply_text)
    except (OSError, IOError, ValueError) as e:
        # Log the error
        print("Error processing the image:", e)
        # Reply to the user with an error message
        error_message = "Error processing the image. Please make sure it is a valid image file in JPEG, PNG, or GIF format."
        bot.reply_to(message, error_message)

@bot.message_handler(func=lambda message: True)
def handle_text(message):
    # Handle text messages if needed
    pass

bot.polling()
